## Cruzamento dos Dados (Enunciado Questões + Microdados)

* **Input**: Local do CSV de Questões Extraídas + Local do CSV de Microdados

* **Output**: CSV com as colunas -> *numero_questao*, *enunciado*, *alternativas*, *nu_param_B* e *gabarito*

In [ ]:
import pandas as pd

In [2]:
# Definição de Variáveis de Input e Output
year = "2020"

question_path = f"../data/extracted_questions/enem_{year}.csv"
microdados_path = f"../data/microdados/microdados_{year}.csv"
output_path = f"../data/merged_data/enem_{year}.csv"

In [3]:
# Ler o CSV extraído do PDF (resultado de Text_Extraction.ipynb)
df_questions = pd.read_csv(question_path, encoding="utf-8", quotechar='"')

# Ler o CSV de microdados
df_microdados = pd.read_csv(microdados_path, delimiter=";", encoding="latin1")

In [12]:
co_prova = {
    "2015": 275,
    "2016": 351,
    "2017": 391,
    "2020": 597,
    "2009": 49,
    "2010": 89,
    "2011": 121
}

In [5]:
# Filtrar os microdados para SG_AREA 'CN', TX_COR 'AZUL' e CO_PROVA 'X'
# ATENÇÃO: O Código da Prova para a Aplicação Regular muda a cada ano
df_microdados_filtrado = df_microdados[
    (df_microdados["SG_AREA"] == "H")  # Ciência das Naturezas
    & (df_microdados["TX_COR"].str.upper() == "AZUL")  # Apenas caderno azul
    & (df_microdados["CO_PROVA"] == co_prova[year])  # Aplicação Regular
]

# Selecionar apenas as colunas de interesse dos microdados
df_microdados_sel = df_microdados_filtrado[
    ["CO_POSICAO", "NU_PARAM_B", "TX_GABARITO"]
].copy()

if year == "2016":
    df_microdados_sel["CO_POSICAO"] = df_microdados_sel["CO_POSICAO"] - 45
elif year == "2017":
    df_microdados_sel["CO_POSICAO"] = df_microdados_sel["CO_POSICAO"] + 90

In [6]:
# Certificar que os tipos das chaves de junção são compatíveis
df_questions["numero_questao"] = df_questions["numero_questao"].astype(str)
df_microdados_sel["CO_POSICAO"] = df_microdados_sel["CO_POSICAO"].astype(str)

# Fazer o cruzamento usando o número da questão (numero_questao e CO_POSICAO)
df_merged = pd.merge(
    df_questions,
    df_microdados_sel,
    left_on="numero_questao",
    right_on="CO_POSICAO",
    how="left",
)

In [ ]:
# Remover a coluna CO_POSICAO (redundante)
df_merged.drop(columns=["CO_POSICAO"], inplace=True)

# Renomeando colunas
df_merged.rename(columns={
    "NU_PARAM_B": "nu_param_B",
    "TX_GABARITO": "gabarito"
}, inplace=True)


# Salvando DF resultante
df_merged.to_csv(output_path, index=False, encoding="utf-8")

print("Merge concluído.")

In [ ]:
df = pd.read_csv(output_path)
df.head()